In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/train_set.csv')

FileNotFoundError: [Errno 2] File b'data/train_set.csv' does not exist: b'data/train_set.csv'

In [128]:
df.columns

Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
       'Claim_Amount'],
      dtype='object')

In [129]:
#function to get number of missing values in a column
def get_na_num(column): #input the whole column
    return column.isnull().sum() + column[column == ''].count() + column[column == '?'].count()

In [130]:
#function to get number of valid values in a column
def get_valid_num(column): #input the whole column
    return len(column) - get_na_num(column)

In [131]:
#function to get minimum value in a column
def get_min(column): #input the whole column
    return min(column)

In [132]:
#function to get maximum value in a column
def get_max(column): #input the whole column
    return max(column)

In [133]:
#function to get mean in a column
def get_mean(column): #input the whole column
    return column.mean()

In [134]:
#function to get std in a column
def get_std(column): #input the whole column
    return column.std()

In [135]:
#function to get skewness in a column
def get_skew(column): #input the whole column
    return column.skew()

In [136]:
#function to get number of distict values in a column
def get_distinct_num(column): #input the whole column
    return len(column.unique().tolist())

In [137]:
#function to get count of each distict value in a column
def get_distinct_count(column): #input the whole column
    if get_distinct_num(column) > 5:
        print('Number of distict values is larger than 5. We stop updating the number of distinct values')
    else:
        return column.value_counts()

In [138]:
#function to get median in a column
import statistics
def get_median(column):
    if get_distinct_num(column) > 5:
        print('Number of distict values is larger than 5. We do not calculate median')
    else:
        return statistics.median(column)

In [139]:
#function to get mode and count for the mode in a column
def get_mode(column):
    return (column.mode()[0],column[column==column.mode()[0]].count())

In [140]:
#function to check whether a numerical column is continuous or discrete
def check_cont_or_dis(column,threshold):
    if get_distinct_num(column)/get_valid_num(column) > float(threshold): #if relative is bigger than the threshold, we consider it to be continuous 
        return ('continuous')
    else:
        return ('discrete')

In [141]:
#function to determine data type of a column
def column_type(column, threshold):
    if ((column.dtype == np.float64 or column.dtype == np.int64) and check_cont_or_dis(column,threshold) == 'continuous'):
        return ('cont num')
    elif ((column.dtype == np.float64 or column.dtype == np.int64) and check_cont_or_dis(column,threshold) == 'discrete'):
        return ('dist num')
    else:
        return ('cat')

In [142]:
#function to do basic variable screening and create basic statistical report
def Stats_Collection(df):
    threshold = input('Threshold to determine continuous or discrete: ')
    for c in df:
        print('Variable name: ',c)
        
        #Basic variable screening
        if get_na_num(df[c])/len(df[c]) > 0.5:
            print('More 50% missing values, drop this column\n')
            df = df.drop(columns=c)
            continue
        if (column_type(df[c],threshold) == 'cont num') and (get_min(df[c]) == get_max(df[c])):
            print('All same value, drop this column\n')
            df = df.drop(columns=c)
            continue
        if (column_type(df[c],threshold) == 'dist num' or column_type(df[c],threshold) == 'cat') and (get_mode(df[c])[1]/get_valid_num(df[c]) > 0.95):
            print('Mode contains more than 95% cases, drop this column\n')
            df = df.drop(columns=c)
            continue
        if (column_type(df[c],threshold) == 'cat') and (get_distinct_num(df[c]) > 100):
            print('More than 100 categories, drop this column\n')
            df = df.drop(columns=c)
            continue
            
        #Basic statistic report
        print('Variable type: ', column_type(df[c],threshold))
        print ('Number of missing values: ',get_na_num(df[c]))
        print ('Number of valid values: ',get_valid_num(df[c]))
        if column_type(df[c],threshold) == 'cont num' or column_type(df[c],threshold) == 'dist num':
            print('Minimum value: ', get_min(df[c]))
            print('Maximum value: ', get_max(df[c]))
            print('Mean: ',get_mean(df[c]))
            print('Standard Deviation: ',get_std(df[c]))
            print('Skewness: ',get_skew(df[c]))
        if column_type(df[c],threshold) == 'cont num':
            print('Number of distinct values: ',get_distinct_num(df[c]))
            print('Number of cases for each distinct value: ')
            print(get_distinct_count(df[c]))
        else:
            print('Number of categories: ', get_distinct_num(df[c]))
            print('The counts of each category: ')
            print(get_distinct_count(df[c]))
            print('Mode: ', get_mode(df[c])[0],'Count: ',get_mode(df[c])[1])    
        print()

In [144]:
df_variable = df.drop(columns = ['Claim_Amount'])
Stats_Collection(df_variable)

Threshold to determine continuous or discrete: 0.9
Variable name:  Row_ID
Variable type:  cont num
Number of missing values:  0
Number of valid values:  13184290
Minimum value:  1
Maximum value:  13184290
Mean:  6592145.5
Standard Deviation:  3805976.8346246025
Skewness:  6.979858407401019e-17
Number of distinct values:  13184290
Number of cases for each distinct value: 
Number of distict values is larger than 5. We stop updating the number of distinct values
None

Variable name:  Household_ID
Variable type:  dist num
Number of missing values:  0
Number of valid values:  13184290
Minimum value:  1
Maximum value:  7542113
Mean:  4128241.6081427974
Standard Deviation:  2248414.572795931
Skewness:  -0.20823358848553647
Number of categories:  4309042
The counts of each category: 
Number of distict values is larger than 5. We stop updating the number of distinct values
None
Mode:  4282380 Count:  37

Variable name:  Vehicle
Variable type:  dist num
Number of missing values:  0
Number of val